In [ ]:
with open("5/input.txt") as f:
    data = f.read()
data

In [ ]:
categories = data.split("\n\n")
categories

In [ ]:
seeds = list(map(lambda seed_str: int(seed_str.strip()), categories[0].replace("seeds: ", "").split(" ")))
seeds

In [ ]:
category_maps = {}

for category in categories[1:]:
    category_map_name, mappings_str = category.split(" map:\n", maxsplit=1)
    from_category, to_category = category_map_name.split("-to-")
    mapping_lines = mappings_str.strip().split("\n")
    from_to_category = []
    for line in mapping_lines:
        destination_range_start, source_range_start, range_length = map(int, line.split())
        source_range_end_inclusive = source_range_start + range_length - 1
        from_to_category.append((source_range_start, source_range_end_inclusive, destination_range_start))
    from_to_category = sorted(from_to_category, key=lambda x: x[0])
    category_maps[from_category] = (to_category, from_to_category)
category_maps

In [ ]:
def find_matching_range_linear(value: int, from_to_map: list[tuple[int, int, int]]) -> tuple[int | None, tuple[int | None, int | None], int]:
    for i, (source_range_start, source_range_end_inclusive, destination_range_start) in enumerate(from_to_map):
        if source_range_start <= value <= source_range_end_inclusive:
            diff_from_source_start = value - source_range_start
            # return destination_range_start + diff_from_source_start
            return i, (source_range_start, source_range_end_inclusive), (destination_range_start + diff_from_source_start)
    return value

def find_matching_range_binary(value: int, from_to_map: list[tuple[int, int, int]]) -> tuple[int | None, tuple[int, int] | None, int]:
    start_index, end_index = 0, len(from_to_map)

    while start_index < end_index:
        mid_index = (end_index + start_index) // 2
        source_range_start, source_range_end_inclusive, destination_range_start = from_to_map[mid_index]

        # print(mid_index)
        # print(source_range_start, source_range_end_inclusive)
        if source_range_start <= value <= source_range_end_inclusive:
            diff_from_source_start = value - source_range_start
            # print("found range")
            # return destination_range_start + diff_from_source_start
            return mid_index, (source_range_start, source_range_end_inclusive), (destination_range_start + diff_from_source_start)
        
        elif value > source_range_end_inclusive:
            start_index = mid_index + 1

        else:
            end_index = mid_index
    return None, (None, None), value


def map_from_to_category(value: int, from_to_map: list[tuple[int, int, int]]):
    *_, mapped_value = find_matching_range_binary(value, from_to_map)
    return mapped_value

In [ ]:
def map_seeds_all_categories(start_values: list[int], category_maps: dict[str, tuple[str, list[tuple[int, int, int]]]]) -> list[int]:
    start_category = "seed"
    mapped_values = start_values
    
    while start_category in category_maps:
        start_category, category_map = category_maps[start_category]
        mapped_values = [map_from_to_category(value, category_map) for value in mapped_values]
    
    return mapped_values


In [ ]:
locations = map_seeds_all_categories(seeds, category_maps)
locations

In [ ]:
min(locations)

## Part 2

In [ ]:
def map_seed_all_categories(value: int, category_maps: dict[str, tuple[str, list[tuple[int, int, int]]]]) -> list[int]:
    start_category = "seed"
    mapped_value = value
    
    while start_category in category_maps:
        start_category, category_map = category_maps[start_category]
        mapped_value = map_from_to_category(value, category_map)
    
    return mapped_value

In [ ]:
seeds

In [ ]:
list(zip(seeds[::2], seeds[1::2]))

In [ ]:
from functools import reduce

def seed_nums(seed_ranges):
    for seed, num_seeds in seed_ranges:
        for seed_number in range(seed, seed + num_seeds):
            yield seed_number

def location_nums(seeds):
    for seed in seeds:
        yield map_seed_all_categories(seed, category_maps)

seed_ranges = list(zip(seeds[::2], seeds[1::2]))
all_seeds = seed_nums(seed_ranges)
all_location_nums = location_nums(all_seeds)
all_location_nums

In [ ]:
min_location = reduce(min, location_nums(seed_nums(seed_ranges[0:1])))
min_location

In [ ]:
seed_start, seed_num = seed_ranges[0]
seed_end = seed_start + seed_num
seed_start